In [45]:
src_dir = '/Volumes/GoogleDrive/My Drive/Projects/NeuSurPlan_2021/IntraOpMap/code/data/derivatives'

In [45]:
src_des = '/home/paolo/Datasets/IntraOpMap/code/data'
out_des = '/home/paolo/Datasets/IntraOpMap/data'
out_gii = '/home/paolo/Datasets/IntraOpMap/data/point_to_surf'

In [43]:
src_des = '/Volumes/paolo/Datasets/IntraOpMap/code/data'
out_des = '/Volumes/paolo/Datasets/IntraOpMap/data'
out_gii = '/Volumes/paolo/Datasets/IntraOpMap/data/point_to_surf'

In [36]:
import os
import csv
import numpy as np
import nibabel as nib
import dipy.tracking.utils as tu

In [75]:
def load_csv(src_csv, hdr=False):
    csv_fd = open(src_csv)
    csv_header = ""
    csv_reader = csv.reader(csv_fd)
    if hdr:
        csv_header = next(csv_reader)
    csv_rows = []
    for row in csv_reader:
        csv_rows.append(row)
    csv_fd.close()
    return csv_header, csv_rows

In [76]:
def split_3d_points_by_roi(src, out, tag='cortical'):
    hdr, row = load_csv(src, hdr=True)
    roi_split = []
    rois = list(set([r[1] for r in row]))
    xyz = {}
    for roi in rois: 
        xyz[roi]=[]
    for r in row:
        xyz[r[1]].append([r[2],r[3],r[4]])
    for roi in rois:
        roi_split.append(os.path.join(out, 'des-%s_roi-%s.csv' % (tag, roi)))
        with open(roi_split[-1], 'w') as out_csv:
            for des in xyz[roi]:
                out_csv.write(",".join(des) + "\n")
            print("%3d %s" % (len(xyz[roi]), roi))
    return roi_split, rois

In [83]:
def split_3d_points_by_hemi(src, out, emi='lh'):
    hdr, row = load_csv(src)
    xyz = []
    for r in row:
        if (emi == 'lh' and float(r[0]) <= 0):
            xyz.append([r[0],r[1],r[2]])
        if (emi == 'rh' and float(r[0]) >= 0):
            xyz.append([r[0],r[1],r[2]])
    with open(out, 'w') as out_csv:
        for des in xyz:
            out_csv.write(",".join(des) + "\n")
        print("Selected points: %3d/%3d" % (len(xyz), len(row)))

In [87]:
def convert_to_node(src, out):
    hdr, row = load_csv(src)
    with open(out, 'w') as out_csv:
        for r in row:
            node = [r[0], r[1], r[2], "1.1", "0.5"]
            out_csv.write(" ".join(node) + "\n")

In [78]:
ref_tag = 'cortical_batch-04'
src_csv = os.path.join(src_des, "des-%s.csv" % ref_tag)
out_dir = os.path.join(out_des, "des_csv")
_, roi_b04 = split_3d_points_by_roi(src_csv, out_dir, ref_tag)

ref_tag = 'cortical_batch-03'
src_csv = os.path.join(src_des, "des-%s.csv" % ref_tag)
out_dir = os.path.join(out_des, "des_csv")
_, roi_b03 = split_3d_points_by_roi(src_csv, out_dir, ref_tag)

ref_tag = 'cortical_batch-02'
src_csv = os.path.join(src_des, "des-%s.csv" % ref_tag)
out_dir = os.path.join(out_des, "des_csv")
_, roi_b02 = split_3d_points_by_roi(src_csv, out_dir, ref_tag)

ref_tag = 'cortical_batch-01'
src_csv = os.path.join(src_des, "des-%s.csv" % ref_tag)
out_dir = os.path.join(out_des, "des_csv")
_, roi_b01 = split_3d_points_by_roi(src_csv, out_dir, ref_tag)

  5 VISUAL
  3 PHONOLOGICAL
  2 ATAXIA
 45 SPEECH_ARREST
 15 SENSORIAL
 17 SEMANTIC
 30 ANOMIA
  4 MOVEMENT_ARREST
  1 DISARTRIA
  2 AMODAL_ANOMIA
  2 COORDINATION
 46 MOTOR
  7 VISUAL
  7 PHONOLOGICAL_APHASIA
332 SPEECH_ARREST
 45 SEMANTIC
161 SENSORIAL
  1 READING
 14 HYPOPHONIA
109 ANOMIA
 47 VOCALISATION
256 VERBAL_APRAXIA
325 MOVEMENT_ARREST
  8 EYES_MOVEMENT
 49 MENTALIZING
 79 AMODAL_ANOMIA
  2 ECHOLALIA
278 MOTOR
 10 VISUO_SPATIAL
  5 PHONOLOGICAL
110 SPEECH_ARREST
 25 SEMANTIC
 53 SENSORIAL
  7 SPATIAL_PERCEPTION
 47 ANOMIA
  3 EYES_MOVEMENT
 81 MOVEMENT_ARREST
 72 VERBAL_APRAXIA
 12 MENTALIZING
 17 AMODAL_ANOMIA
131 MOTOR
  1 READING
  7 PHONOLOGICAL
131 SPEECH_ARREST
 11 SEMANTIC
 92 SENSORIAL
 16 SPATIAL_PERCEPTION
 39 ANOMIA
  1 EYES_MOVEMENT
 58 VERBAL_APRAXIA
178 MOTOR
  1 READING


In [79]:
ref_tag = 'subcortical_batch-01'
src_csv = os.path.join(src_des, "des-%s.csv" % ref_tag)
out_dir = os.path.join(out_des, "des_csv")
_, roi_b01 = split_3d_points_by_roi(src_csv, out_dir, ref_tag)

ref_tag = 'subcortical_batch-02'
src_csv = os.path.join(src_des, "des-%s.csv" % ref_tag)
out_dir = os.path.join(out_des, "des_csv")
_, roi_b02 = split_3d_points_by_roi(src_csv, out_dir, ref_tag)

ref_tag = 'subcortical_batch-03'
src_csv = os.path.join(src_des, "des-%s.csv" % ref_tag)
out_dir = os.path.join(out_des, "des_csv")
_, roi_b03 = split_3d_points_by_roi(src_csv, out_dir, ref_tag)

ref_tag = 'subcortical_batch-04'
src_csv = os.path.join(src_des, "des-%s.csv" % ref_tag)
out_dir = os.path.join(out_des, "des_csv")
_, roi_b04 = split_3d_points_by_roi(src_csv, out_dir, ref_tag)

 10 VISUAL
 42 PHONOLOGICAL
  4 MOVEMENT_VERBAL_ARREST
 23 SPEECH_ARREST
 64 SEMANTIC
 45 SENSORIAL
  6 READING
  9 INVOLUNTARY_MOVEMENTS
  5 ANOMIA
  2 EYES_MOVEMENT
 25 VERBAL_APRAXIA
  3 MOVEMENT_ARREST
  7 AMODAL_ANOMIA
 75 MOTOR
  8 VISUO_SPATIAL
 12 PERSEVERATIONS
 10 VISUAL
  5 SPEECH_ARREST
  1 INVOLUNTARY_MOVEMENTS
  2 VERTIGO
  3 VISUO_SPATIAL
 26 SENSORIAL
  5 EYES_MOVEMENT
 16 VERBAL_APRAXIA
 27 MOTOR
 26 PHONOLOGICAL
 69 SEMANTIC
  1 VERBAL_MOTOR_APRAXIA
  2 PSYCHIATRIC_REACTIONS
 14 MENTALIZING
 20 AMODAL_ANOMIA
  7 READING
 26 MOVEMENT_VERBAL_ARREST
 12 PERSEVERATIONS
 19 ANOMIA
 52 MOVEMENT_ARREST
  2 ACOUSTIC_RESPONSES
 29 VISUAL
 37 PHONOLOGICAL
 72 SPEECH_ARREST
  3 ACOUSTIC_RESPONSES
 38 SENSORIAL
140 SEMANTIC
  7 VERTIGO
  1 READING
 41 ANOMIA
  5 EYES_MOVEMENT
 70 MOVEMENT_ARREST
 33 VERBAL_APRAXIA
 64 MENTALIZING
 36 MOTOR
  6 VISUO_SPATIAL
 22 PERSEVERATIONS
 13 VISUAL
  2 PHONOLOGICAL
  7 SEMANTIC
  8 SENSORIAL
  2 VISUAL_RECOGNITION
  1 APRAXIA
 14 ANOMIA
  6 

In [96]:
ref_tag = "ANOMIA"
b01_src = os.path.join(src_dir, "des_csv/des-cortical_batch-01_roi-%s.csv" % ref_tag)
b02_src = os.path.join(src_dir, "des_csv/des-cortical_batch-02_roi-%s.csv" % ref_tag)
b03_src = os.path.join(src_dir, "des_csv/des-cortical_batch-03_roi-%s.csv" % ref_tag)
csv_out = os.path.join(src_dir, "des_csv/des-cortical_roi-%s.csv" % ref_tag)
cmd = "cat \'%s\' \'%s\' \'%s\' >> \'%s\'" % (b01_src, b02_src, b03_src, csv_out)
os.system(cmd)


0

In [174]:
ref_tag = "ANOMIA"
ref_des = 'cortical'
ref_emi = 'lh'
src_csv = os.path.join(src_dir, "des_csv/des-%s_roi-%s.csv" % (ref_des, ref_tag))
out_csv = os.path.join(src_dir, "des_csv/des-%s_roi-%s_%s.csv" % (ref_des, ref_tag, ref_emi))
split_3d_points_by_hemi(src_csv, out_csv, ref_emi)

Selected points: 192/202


In [80]:
ref_tag = "SEMANTIC"
b01_src = os.path.join(out_des, "des_csv/des-cortical_batch-01_roi-%s.csv" % ref_tag)
b02_src = os.path.join(out_des, "des_csv/des-cortical_batch-02_roi-%s.csv" % ref_tag)
b03_src = os.path.join(out_des, "des_csv/des-cortical_batch-03_roi-%s.csv" % ref_tag)
b04_src = os.path.join(out_des, "des_csv/des-cortical_batch-04_roi-%s.csv" % ref_tag)
csv_out = os.path.join(out_des, "des_csv/des-cortical_roi-%s.csv" % ref_tag)
cmd = "cat \'%s\' \'%s\' \'%s\' \'%s\' >> \'%s\'" % (b01_src, b02_src, b03_src, b04_src, csv_out)
os.system(cmd)

0

In [84]:
ref_tag = "SEMANTIC"
ref_des = 'cortical'
ref_emi = 'lh'
src_csv = os.path.join(out_des, "des_csv/des-%s_roi-%s.csv" % (ref_des, ref_tag))
out_csv = os.path.join(out_des, "des_csv/des-%s_roi-%s_%s.csv" % (ref_des, ref_tag, ref_emi))
split_3d_points_by_hemi(src_csv, out_csv, ref_emi)

Selected points: 271/282


In [88]:
ref_tag = "SEMANTIC"
ref_des = 'cortical'
ref_emi = 'lh'
src_csv = os.path.join(out_des, "des_csv/des-%s_roi-%s_%s.csv" % (ref_des, ref_tag, ref_emi))
out_csv = os.path.join(out_des, "des_csv/des-%s_roi-%s_%s.node" % (ref_des, ref_tag, ref_emi))
convert_to_node(src_csv, out_csv)

In [89]:
def cloud2nifti(src, ref, out, emi=''):
    
    img = nib.load(ref)
    vox_size = img.header.get_zooms()
    shape = img.header.get_data_shape()
    affine = img.affine
    header = img.header
    linT, offset = tu._mapping_to_voxel(affine)
    
    cloud = np.loadtxt(src, delimiter=',')
    
    mask = np.zeros(shape, dtype=np.float)
    stem = tu._to_voxel_coordinates(cloud[:,:3], linT, offset)
    i, j, k = stem.T
    if emi == 'lh':
        mask[i>90, j, k] = 1.0
    if emi == 'lh':
        mask[i<92, j, k] = 1.0
    if emi == '':
        mask[i, j, k] = 1.0
    
    nii = nib.Nifti1Image(mask, affine, header)
    nii.update_header()
    nib.save(nii, out)
    
    return stem

In [149]:
lab = 'ANOMIA'
emi = 'lh'
src = '../data/derivatives/des_csv/des-cortical_roi-%s.csv' % lab
ref = '../metadata/MNI152/MNI152_T1_brain.nii.gz'
out = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.nii.gz' % (lab, emi)


In [90]:
lab = 'SEMANTIC'
emi = 'lh'
src = os.path.join(out_des, 'des_csv/des-cortical_roi-%s_%s.csv' % (lab, emi))
ref = os.path.join(src_des, '../metadata/MNI152/MNI152_T1_brain.nii.gz')
out = os.path.join(out_gii, 'des-cortical_roi-%s_%s.nii.gz' % (lab, emi))


In [92]:
cloud2nifti(src, ref, out)

array([[151, 142,  90],
       [162, 100,  87],
       [155, 147,  79],
       [161, 114,  55],
       [158,  85,  84],
       [159,  92,  76],
       [156, 113,  76],
       [146, 160, 100],
       [159, 109,  77],
       [141, 153,  75],
       [160,  88,  88],
       [128, 138, 139],
       [150, 155,  82],
       [160,  94,  79],
       [137, 147, 114],
       [156, 100,  84],
       [144, 150,  83],
       [155,  89,  78],
       [154,  85,  77],
       [150, 142,  85],
       [147, 147,  90],
       [150, 145,  78],
       [153, 135,  64],
       [157, 113,  76],
       [146, 146,  97],
       [147, 149,  86],
       [158, 107,  72],
       [158, 103,  72],
       [157, 108,  81],
       [159, 100,  61],
       [136, 137, 124],
       [131, 133, 130],
       [136, 138, 132],
       [138, 144, 100],
       [148, 150,  89],
       [146, 158,  89],
       [144, 164,  80],
       [138, 137, 121],
       [144, 133, 110],
       [134, 129, 127],
       [137, 158,  78],
       [137, 159

In [162]:
lab = 'VERBAL_APRAXIA'
emi = "lh"
src_vol = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.nii.gz' % (lab, emi)
out_gii = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s_sfwhm-10.gii' % (lab, emi)

In [28]:
lab = 'SEMANTIC'
emi = "lh"
src_vol = os.path.join(out_gii, 'des-cortical_roi-%s_%s.nii.gz' % (lab, emi))
out_gii = os.path.join(out_gii, 'des-cortical_roi-%s_%s_sfwhm-10.gii' % (lab, emi))

In [29]:
cmd = "mri_vol2surf --mov %s --surf pial --mni152reg --out %s --hemi lh --surf-fwhm 10" % (src_vol, out_gii)
cmd
#os.system(cmd)

'mri_vol2surf --mov /Volumes/paolo/Datasets/IntraOpMap/data/point_to_surf/des-cortical_roi-SEMANTIC_lh.nii.gz --surf pial --mni152reg --out /Volumes/paolo/Datasets/IntraOpMap/data/point_to_surf/des-cortical_roi-SEMANTIC_lh_sfwhm-10.gii --hemi lh --surf-fwhm 10'

In [120]:
lab = 'ANOMIA'
emi = "lh"
vol = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.nii.gz' % (lab, emi)
surf = "lh.pial.asc"
projtype = "1"
projdist = "10"
projmap = "1"
reg = "mni2reg"
vsm = "novsm"
interp = "1"
out = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.gii' % (lab, emi)
cmd = "mri_vol2surf --vol2surf %s %s %s %s %s %s %s %s %s" % \
    (vol, surf, projtype, projdist, projmap, reg, vsm, interp, out)
cmd

'mri_vol2surf --vol2surf ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.nii.gz lh.pial.asc 1 10 1 mni2reg novsm 1 ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii'

In [101]:
os.system(cmd)


srcvol = ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.nii.gz
srcreg = /Applications/freesurfer/7.1.1/average/mni152.register.dat
srcregold = 0
srcwarp unspecified
surf = pial
hemi = lh
reshape = 0
interp = nearest
float2int = round
GetProjMax = 0
INFO: float2int code = 0
niiRead(): NIFTI_UNITS_UNKNOWN, assuming mm
niiRead(): detected input as 64 bit double, reading in as 32 bit float
Done loading volume
INFO: This REGISTER_DAT transform is valid only for volumes between  COR types with c_(r,a,s) = 0.
Input reg is register.dat
-------- original matrix -----------
 0.99753  -0.00732   0.01760   0.95709;
-0.01296  -0.00926   0.99706  -17.81596;
-0.01460  -1.00094   0.00244  -18.54964;
 0.00000   0.00000   0.00000   1.00000;
-------- original matrix -----------
INFO: smoothing volume at fwhm = 20 mm (std = 8.49322)
Reading surface /Applications/freesurfer/7.1.1/subjects/fsaverage/surf/lh.pial
Done reading source surface
Mapping Source Volume onto Source Subject Surface
 1 0

0

In [2]:
lab = 'ANOMIA'
emi = "lh"
ref_surf = '/Volumes/paolo/Downloads/point_to_surf/lh.pial.asc'
src_vol = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.nii.gz' % (lab, emi)
out_gii = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s_fsl.gii' % (lab, emi)

In [33]:
lab = 'SEMANTIC'
emi = "lh"
ref_surf = os.path.join(out_gii, 'lh.pial.asc')
src_vol = os.path.join(out_gii, 'des-cortical_roi-%s_%s.nii.gz' % (lab, emi))
out_surf = os.path.join(out_gii, 'des-cortical_roi-%s_%s_surf.gii' % (lab, emi))

In [34]:
cmd = "surf_proj  --data=%s --surf=%s --out=%s --surfout" % (src_vol, ref_surf, out_surf)
cmd

'surf_proj  --data=/Volumes/paolo/Datasets/IntraOpMap/data/point_to_surf/des-cortical_roi-SEMANTIC_lh.nii.gz --surf=/Volumes/paolo/Datasets/IntraOpMap/data/point_to_surf/lh.pial.asc --out=/Volumes/paolo/Datasets/IntraOpMap/data/point_to_surf/des-cortical_roi-SEMANTIC_lh_surf.gii --surfout'

In [104]:
os.system(cmd)

read data
read surface
project data
save
numda2 3
write vertcies 163842
write traingles 327680
327680 163842
resize trinagle array
write faces 983040
scalar 0 2
set scalar names 1 MyScalars
do memcpy 
scalar int 0
done scalar
null


0

In [114]:
lab = 'ANOMIA'
emi = "lh"
src_gii = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.gii' % (lab, emi)
out_stl = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.stl' % (lab, emi)

In [115]:
cmd = "mris_convert %s %s" % (src_gii, out_stl)
cmd

'mris_convert ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.stl'

In [116]:
os.system(cmd)

mriseadGIFTIfile: mris is NULL! found when parsing file ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii
error: Invalid argument
error: mris_convert: could not read surface file ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii 


65280

In [156]:
%cd '/Volumes/GoogleDrive/My Drive/Projects/NeuSurPlan_2021/IntraOpMap/code/preprocessing'

/Volumes/GoogleDrive/My Drive/Projects/NeuSurPlan_2021/IntraOpMap/code/preprocessing


In [157]:
pwd

'/Volumes/GoogleDrive/My Drive/Projects/NeuSurPlan_2021/IntraOpMap/code/preprocessing'

In [83]:
ls ../data/derivatives/point_to_surf

MNI152_T1_1mm_brain.stl*
MNI152_T1_1mm_brain_lh.gii*
MNI152_T1_1mm_brain_lh.stl*
MNI152_T1_1mm_brain_lh_hole.ply*
des-cortical_batch-03_roi-MOTOR_lh.csv*
des-cortical_batch-03_roi-MOTOR_lh.gii*
des-cortical_batch-03_roi-MOTOR_lh.nii.gz*
des-cortical_batch-03_roi-MOTOR_lh.ply*
des-cortical_batch-03_roi-MOTOR_lh_fwhm-10.gii*
des-cortical_batch-03_roi-MOTOR_lh_fwhm-20.gii*
des-cortical_roi-SPEECH_ARREST_lh.nii.gz*
des-cortical_roi-VERBAL_APRAXIA_lh.nii.gz*
orig.nii.gz*
rh.MNI152_T1_1mm_brain.gii*


In [112]:
ls ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii

../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii*


In [132]:
img = nib.load('../metadata/MNI152/MNI152_T1_1mm.nii.gz')


In [141]:
img.header.set_xyzt_units("mm", "sec")

In [134]:
img.header.get_dim_info()

(None, None, None)

In [135]:
img.header.get_data_dtype()

dtype('<f4')

In [142]:
img.header.get_xyzt_units()

('mm', 'sec')

AttributeError: 'tuple' object has no attribute 'dtype'

In [147]:
nib.Nifti1Image?

In [151]:
import dipy.stats.analysis as dsa

In [152]:
dsa.gaussian_weights?

In [168]:
int("31")

31